In [7]:
import nltk
nltk.download([
"names",
"stopwords",
"state_union",
"twitter_samples",
"movie_reviews",
"averaged_perceptron_tagger",
"vader_lexicon",
"punkt",
... ])
from nltk.tokenize import word_tokenize
import pandas as pd


data = pd.read_csv("data.csv")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yashbhutoria/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
data.head()

,title,date,person,link,transcript
0,"Remarks at Old South Meeting House in Boston, ...",2019-12-31T00:00:00+00:00,Elizabeth Warren,/documents/remarks-old-south-meeting-house-bos...,"\n[As prepared for delivery.]\nIn a few hours,..."
1,Steyer Campaign Press Release - Tom Steyer Rea...,2019-12-31T00:00:00+00:00,Tom Steyer,/documents/steyer-campaign-press-release-tom-s...,"\n(SAN FRANCISCO, December 31, 2019) — Today, ..."
2,Biden Campaign Press Release - Biden for Presi...,2019-12-31T00:00:00+00:00,Joseph R. Biden,/documents/biden-campaign-press-release-biden-...,\nCampaign Continues Growing Super Tuesday Inf...
3,Article by Mike Bloomberg - You're In Charge: ...,2019-12-31T00:00:00+00:00,Michael Bloomberg,/documents/article-mike-bloomberg-youre-charge...,\nThis article was written by Mike Bloomberg.\...
4,Trump Campaign Statement - Happy New Year from...,2019-12-31T00:00:00+00:00,Donald J. Trump,/documents/trump-campaign-statement-happy-new-...,"\nAs we ring in a New Year, Brad Parscale, Cam..."


In [10]:
tokens = word_tokenize(data.loc[0,"transcript"]) 

In [12]:
# Example using AFINN lexicon
from afinn import Afinn

afinn = Afinn()
sentiment_score = sum([afinn.score(word) for word in tokens])


In [17]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(data.loc[0,"transcript"])

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - '/Users/yashbhutoria/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/share/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [15]:
tokens[2]

'prepared'

In [18]:
print(data.loc[0,"transcript"])


[As prepared for delivery.]
In a few hours, the dawn of a New Year will break over America, offering us a moment for pause and reflection.
It is normally a moment for optimism. But let's face it: This year in America has been anything but normal.
In the past 12 months, the president has become bolder with his lies and more brazen in his law-breaking. He tried to squeeze a foreign government to advance his own personal political fortunes.
Meanwhile, the Republicans in Congress have turned into fawning, spineless defenders of his crimes.
It brought no one any joy, but the House Democrats upheld their sworn duty to the Constitution and impeached the president of the United States. Soon I will return to the Senate to do my sworn duty as well.
But unless some Senate Republicans choose truth over politics, Donald Trump will be emboldened to try to cheat his way through yet another election.
So as we turn the page tomorrow — into an election year and a new decade — there is a chill of fear i